In [ ]:
import cv2
import tensorflow as tf
import os
import keras
import csv
import random
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras.initializers import RandomNormal
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Sat Dec  3 18:46:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    32W /  70W |   7058MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
num_classes = 3
img_rows, img_cols = 256, 256
batch_size = 30
num_epochs = 20

In [ ]:
model = Sequential()
model.add(Conv2D(8, 3, padding='same', activation='relu', input_shape=(img_rows, img_cols,3)))
model.add(Conv2D(8, 3, padding='same', activation='relu', input_shape=(img_rows, img_cols,3)))
model.add(MaxPooling2D())
model.add(Conv2D(16, 3, padding='same', activation='relu'))
model.add(Conv2D(16, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(Conv2D(128, 3, padding='same', activation='relu'))
model.add(Conv2D(128, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(256, 3, padding='same', activation='relu'))
model.add(Conv2D(256, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(512, 3, padding='same', activation='relu'))
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_130 (Conv2D)         (None, 256, 256, 8)       224       
                                                                 
 conv2d_131 (Conv2D)         (None, 256, 256, 8)       584       
                                                                 
 max_pooling2d_59 (MaxPoolin  (None, 128, 128, 8)      0         
 g2D)                                                            
                                                                 
 conv2d_132 (Conv2D)         (None, 128, 128, 16)      1168      
                                                                 
 conv2d_133 (Conv2D)         (None, 128, 128, 16)      2320      
                                                                 
 max_pooling2d_60 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                

In [ ]:
model_input = model.inputs
flatten_layer = Flatten()(model.output)
dense_layer=Dense(512, activation='relu')(flatten_layer)
output_layer = Dense(num_classes, activation='softmax',kernel_initializer=RandomNormal(mean=0.0, stddev=0.001))(dense_layer)
model = Model(inputs=model_input, outputs=output_layer)
model.compile(optimizer=Nadam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_130_input (InputLaye  [(None, 256, 256, 3)]    0         
 r)                                                              
                                                                 
 conv2d_130 (Conv2D)         (None, 256, 256, 8)       224       
                                                                 
 conv2d_131 (Conv2D)         (None, 256, 256, 8)       584       
                                                                 
 max_pooling2d_59 (MaxPoolin  (None, 128, 128, 8)      0         
 g2D)                                                            
                                                                 
 conv2d_132 (Conv2D)         (None, 128, 128, 16)      1168      
                                                                 
 conv2d_133 (Conv2D)         (None, 128, 128, 16)      2320

In [ ]:
model_name = 'CNN'
filepath="/content/drive/MyDrive/BioProject/models1/%s-{epoch:02d}-{val_accuracy:.4f}.hdf5"%model_name
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')
callbacks_list = [checkpoint]


In [ ]:
train_datagen = ImageDataGenerator( rescale = 1./255,shear_range = 0.05, zoom_range = 0.05, horizontal_flip = True, vertical_flip = True, rotation_range = 360, width_shift_range = 0.05, height_shift_range = 0.05)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
val_df=pd.read_csv("/content/drive/MyDrive/BioProject/All_data/val_data.csv")
for trainset in range(1,9):
  train_df =pd.read_csv("/content/drive/MyDrive/BioProject/All_data/Train_data/train{}.csv".format(trainset)) 
  train_generator = train_datagen.flow_from_dataframe(
        dataframe=train_df,
        directory='/content/drive/MyDrive/BioProject/All_data/All',
        x_col="filename",
        y_col="class",
        target_size=(img_rows, img_cols),
        batch_size=30,
        class_mode='categorical',shuffle=True)
  validation_generator = test_datagen.flow_from_dataframe(
          dataframe=val_df,
          directory='/content/drive/MyDrive/BioProject/All_data/All',
          x_col="filename",
          y_col="class",
          target_size=(img_rows, img_cols),
          batch_size=30,
          class_mode='categorical',shuffle=True)
  model.fit_generator(train_generator, epochs=num_epochs,validation_data=validation_generator,shuffle=True,callbacks=callbacks_list)

Found 1433 validated image filenames belonging to 3 classes.
Found 845 validated image filenames belonging to 3 classes.


<ipython-input-66-210d89443420>:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=num_epochs,validation_data=validation_generator,shuffle=True,callbacks=callbacks_list) #start training


Epoch 1/20
48/48 [==============================] - 52s 1s/step - loss: 1.0993 - accuracy: 0.3405 - val_loss: 1.0848 - val_accuracy: 0.4734
Epoch 2/20
48/48 [==============================] - 46s 966ms/step - loss: 1.0974 - accuracy: 0.3475 - val_loss: 1.0631 - val_accuracy: 0.4734
Epoch 3/20
48/48 [==============================] - 47s 975ms/step - loss: 1.0857 - accuracy: 0.3608 - val_loss: 0.9561 - val_accuracy: 0.5680
Epoch 4/20
48/48 [==============================] - 47s 993ms/step - loss: 1.0399 - accuracy: 0.4941 - val_loss: 0.9345 - val_accuracy: 0.6249
Epoch 5/20
48/48 [==============================] - 48s 990ms/step - loss: 0.9873 - accuracy: 0.5415 - val_loss: 0.9422 - val_accuracy: 0.5704
Epoch 6/20
48/48 [==============================] - 47s 971ms/step - loss: 0.9618 - accuracy: 0.5576 - val_loss: 1.0596 - val_accuracy: 0.4781
Epoch 7/20
48/48 [==============================] - 47s 972ms/step - loss: 0.9504 - accuracy: 0.5611 - val_loss: 0.8713 - val_accuracy: 0.6320
Ep

In [ ]:
reports={}
test_df = pd.read_csv("/content/drive/MyDrive/BioProject/all_test.csv")
validation_generator = test_datagen.flow_from_dataframe( 
        dataframe=test_df,
        directory='/content/drive/MyDrive/BioProject/All_data/All',
        x_col="filename",
        y_col="class",
        target_size=(256, 256),
        batch_size=20,
        class_mode='categorical',shuffle=True)

Found 7334 validated image filenames belonging to 3 classes.


In [ ]:
trained_models=[]
for r,d,f in os.walk('/content/drive/MyDrive/BioProject/models1'):
  for file in f:
    trained_models.append(os.path.join(r,file))

In [ ]:
print(trained_models)

['/content/drive/MyDrive/BioProject/models1/CNN-01-0.8355.hdf5']

In [ ]:
for trained_model in trained_models:
  k.clear_session()
  net=keras.models.load_model(trained_model)
  covid_label= validation_generator.class_indices['COVID-19']
  pneu_label= validation_generator.class_indices['pneumonia']
  normal_label= validation_generator.class_indices['normal'] 
  tp=0 
  fp=0 
  anum=0 
  ###########
  wrong_covid=0
  correct_covid=0
  not_detected_covid=0
  covid_num=0
  ###########
  wrong_pneu=0
  correct_pneu=0
  not_detected_pneu=0
  pneu_num=0
  ############
  wrong_normal=0
  correct_normal=0
  not_detected_normal=0
  normal_num=0
  ##############
  wrong_covid_normal=0 
  wrong_covid_pneu=0   
  wrong_pneu_covid=0   
  wrong_pneu_normal=0 
  wrong_normal_pneu=0 
  wrong_normal_covid=0  
  ################
  for num,img_name in enumerate(validation_generator.filenames): 
    gt_ind=validation_generator.classes[num] 
    img=cv2.resize(cv2.imread('/content/drive/MyDrive/BioProject/All_data/All/'+img_name),(256,256)) 
    img=img.astype('float32') / 255.0 
    pred_ind=np.argmax(net.predict(np.expand_dims(img,axis=0))[0]) 
    anum+=1 

    if gt_ind==covid_label:
      covid_num+=1
    if gt_ind==pneu_label:
      pneu_num+=1
    if gt_ind==normal_label:
      normal_num+=1
    if gt_ind==covid_label and pred_ind==covid_label: 
      correct_covid+=1
    if gt_ind==covid_label and pred_ind!=covid_label:
      not_detected_covid+=1
      if pred_ind==pneu_label:
        wrong_covid_pneu+=1
      elif pred_ind==normal_label:
        wrong_covid_normal+=1
    if gt_ind!=covid_label and pred_ind==covid_label:
      wrong_covid+=1

    if gt_ind==normal_label and pred_ind==normal_label: 
      correct_normal+=1
    if gt_ind==normal_label and pred_ind!=normal_label:
      not_detected_normal+=1
      if pred_ind==pneu_label:
        wrong_normal_pneu+=1
      elif pred_ind==covid_label:
        wrong_normal_covid+=1
    if gt_ind!=normal_label and pred_ind==normal_label:
      wrong_normal+=1
    ###########################################
    if gt_ind==pneu_label and pred_ind==pneu_label: 
      correct_pneu+=1
    if gt_ind==pneu_label and pred_ind!=pneu_label:
      not_detected_pneu+=1
      if pred_ind==normal_label:
        wrong_pneu_normal+=1
      elif pred_ind==covid_label:
        wrong_pneu_covid+=1
    if gt_ind!=pneu_label and pred_ind==pneu_label:
      wrong_pneu+=1
    ###########################################

    if pred_ind==gt_ind:
      tp+=1
    else:
      fp+=1
  reports[trained_model]=[correct_covid,wrong_covid,not_detected_covid,
                   correct_pneu,wrong_pneu,not_detected_pneu,
                   correct_normal,wrong_normal,not_detected_normal,
                   covid_num,pneu_num,normal_num,tp,fp,wrong_covid_normal,wrong_covid_pneu,
                   wrong_pneu_covid,wrong_pneu_normal,wrong_normal_pneu,wrong_normal_covid]
  print(trained_model)
  print('correct_covid: ',correct_covid/covid_num,'%')
  print('wrong_covid: ',wrong_covid)
  print('not_detected_covid: ',not_detected_covid/covid_num,'%')

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 17ms/step
/content/drive/MyDrive/BioProject/models1/CNN-01-0.8355.hdf5
correct_covid:  0.8222222222222222 %
wrong_covid:  183
not_detected_covid:  0.17777777777777778 %


In [ ]:
temp = 'CNN'

In [ ]:
with open('/content/drive/MyDrive/results/{}-fold{}.csv'.format(temp,1),newline='', mode='w') as csvfile: 
     csv_writer = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
     csv_writer.writerow(['correct_covid','wrong_covid','not_detected_covid',
                          'correct_pneu','wrong_pneu','not_detected_pneu',
                          'correct_normal','wrong_normal','not_detected_normal',
                          'covid_num','pneu_num','normal_num','tp','fp','wrong_covid_normal','wrong_covid_pneu',
                          'wrong_pneu_covid','wrong_pneu_normal','wrong_normal_pneu','wrong_normal_covid'])
     for key in reports.keys():
         row=[key]
         csv_writer.writerow(reports[key])